### Push Code to AMAZON ECR

In [1]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-simple-moviesrecommender

cd container

chmod +x collaborative_filtering/train
chmod +x collaborative_filtering/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  2.292MB
Step 1/11 : FROM ubuntu:16.04
16.04: Pulling from library/ubuntu
3386e6af03b0: Pulling fs layer
49ac0bbe6c8e: Pulling fs layer
d1983a67e104: Pulling fs layer
1a0f3a523f04: Pulling fs layer
1a0f3a523f04: Waiting
49ac0bbe6c8e: Verifying Checksum
49ac0bbe6c8e: Download complete
d1983a67e104: Verifying Checksum
d1983a67e104: Download complete
1a0f3a523f04: Verifying Checksum
1a0f3a523f04: Download complete
3386e6af03b0: Verifying Checksum
3386e6af03b0: Download complete
3386e6af03b0: Pull complete
49ac0bbe6c8e: Pull complete
d1983a67e104: Pull complete
1a0f3a523f04: Pull complete
Digest: sha256:181800dada370557133a502977d0e3f7abda0c25b9bbb035f199f5eb6082a114
Status: Downloaded newer image for ubuntu:16.04
 ---> c6a43cd4801e
Step 2/11 : MAINTAINER Hani <hani.mousli@gmail.com>
 ---> Running in b4e250f4e9a4
Removing intermediate container b4e250f4e9a4
 ---> 9c45aec02076
Step 3/11 : RUN apt-get -y update && apt-get install -y --no

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



### Upload data to S3 bucket

In [4]:
# S3 prefix
prefix = 'MovieLensDataset'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role


import sagemaker as sage
from time import gmtime, strftime

role = get_execution_role()

sess = sage.Session()
WORK_DIRECTORY = 'data'
bucket='sagemakeroct10test'

data_location = sess.upload_data(WORK_DIRECTORY, bucket=bucket,key_prefix=prefix)
data_location

's3://sagemakeroct10test/MovieLensDataset'

### Use ECR Image to train on MovieLens 

In [8]:
algorithm_name='sagemaker-simple-moviesrecommender'

account = sess.boto_session.client('sts').get_caller_identity()['Account'] 
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region,algorithm_name)

cf_model = sage.estimator.Estimator(image,
                       role, 1, 'ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)
cf_model.set_hyperparameters(metric='cosine')

cf_model.fit(data_location)

2020-01-01 16:41:15 Starting - Starting the training job...
2020-01-01 16:41:16 Starting - Launching requested ML instances......
2020-01-01 16:42:21 Starting - Preparing the instances for training...
2020-01-01 16:43:04 Downloading - Downloading input data
2020-01-01 16:43:04 Training - Downloading the training image...
2020-01-01 16:43:39 Uploading - Uploading generated training model
2020-01-01 16:43:39 Completed - Training job completed
Starting the training....
/opt/program/train:67: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  movie_similarity = 1 - pairwise_distances( ratings_matrix.as_matrix(), metric=metric ) #[\u2018cityblock\u2019, \u2018cosine\u2019, \u2018euclidean\u2019, \u2018l1\u2019, \u2018l2\u2019, \u2018manhattan\u2019]
Training completed .....
Training seconds: 51
Billable seconds: 51


### Deploy model 

In [9]:
from sagemaker.predictor import csv_serializer
predictor = cf_model.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

--------------------------------------------------------------------------------------!

### Invoke endpoint to get prediction

In [12]:
# Get prediction for two movies. Each will return top 2 similar movies
print(predictor.predict(np.asarray([1,10])).decode('utf-8'))

161,Top Gun (1986),01-Jan-1986,0.6235435842660795
385,True Lies (1994),01-Jan-1994,0.617273683459067
22,Braveheart (1995),16-Feb-1996,0.6491615111196681
12,"Usual Suspects, The (1995)",14-Aug-1995,0.6267298386704434



In [ ]:
sess.delete_endpoint(predictor.endpoint)